In [1]:
# Prérequis :
# - les fiches métiers au format JSON dans un dossier fiches_metiers
# - Fichier JSON de l'arborescences des activités
# - Fichier JSON de l'arborescence des compétences
# - Fichier CSV des code_rome des métiers et des clusters (obtenus sur R)

# Ce script permet de créer un fichier csv avec :
# - code_rome en clés
# - un dictionnaire en valeurs comprenant :
# --> la liste des appellations du métier
# --> la liste des features (ou tags ou variables)
# --> la liste  métiers du même cluster
import os
import json
import pandas as pd
import numpy as np

In [2]:
# Préparation des chemins utiles
# Main directory
dir_path = '/home/timothee/PycharmProjects/ImpalaPoleEmploi2'
# Fiches métiers
fiches_dir_path = os.path.join(dir_path, 'fiches_metiers')
# Fichier JSON de l'arborescences des activités
act_json = os.path.join(dir_path,'fichier_donnees/activites.json')
# Fichier JSON de l'arborescence des compétences
comp_json = os.path.join(dir_path,'fichier_donnees/competences.json')
# Fichier CSV des code_rome des métiers et des clusters (obtenus sur R)
PE_cluster = os.path.join(dir_path,'Impala/Clustering/jobs_clusters.csv')

# Fichier de sortie :jobs_ifs.json (jobs : intitulés, features, similar jobs)
output_file = os.path.join(dir_path,'Impala/jobs_ifs.json')

In [3]:
# Liste des métiers
list_files = os.listdir(fiches_dir_path)
n = len(list_files)

In [4]:
# Chargement des données de compétences dans la variable comp
with open(comp_json) as json_data:
    comp = json.load(json_data)
    json_data.close()
    
# Chargement des données de d'activités dans la variable act
with open(act_json) as json_data:
    act = json.load(json_data)
    json_data.close()
    
# Chargement des code rome des métiers et de leur cluster
with open(PE_cluster) as csv_data:
    clusters = pd.read_csv(PE_cluster)

In [5]:
# Fonction qui permet de récupérer une liste des code ogr des activités et compétences
# qui décrvient le métier donné en argument
def get_job_features_ogr(file):
    with open(file, 'r') as f:
        job = json.load(f)
    
    job_features = []    
    for item in job["activites_de_base"]:
        job_features.append(int(item['code_ogr']))
    for item in job["savoir_theorique_et_proceduraux"]:
        job_features.append(int(item['code_ogr']))
    for item in job["activite_specifique"]:
        job_features.append(int(item['code_ogr']))
    for item in job["savoir_action"]:
        job_features.append(int(item['code_ogr']))
    return job_features

In [6]:
# Fonction qui permet de récupérer la liste des appelations du métier en argument (fiche métier)
def get_job_intitule(file):
    with open(file, 'r') as f:
        job = json.load(f)
        
    job_intitule = []        
    for item in job["appelations"]:
        job_intitule.append(item["libelle"])
    
    return job_intitule

In [7]:
# Fonction qui permet de récupérer les features de chaque métier
def get_job_features (file):
    with open(file, 'r') as f:
        job = json.load(f)
        
    job_features = []
    job_features_ogr = get_job_features_ogr(file)
    for niv1 in comp:
        for niv2 in comp[niv1]:
            for niv3 in comp[niv1][niv2]:
                for niv4 in comp[niv1][niv2][niv3].values():
                    for item in niv4:         
                        if int(item[1]) in job_features_ogr:
                            job_features.append(item[0])
    for niv1 in act:
        for niv2 in act[niv1]:
            for niv3 in act[niv1][niv2]:
                for niv4 in act[niv1][niv2][niv3]:
                    for niv5 in act[niv1][niv2][niv3][niv4].values():
                        for item in niv5:         
                            if int(item[1]) in job_features_ogr:
                                job_features.append(item[0])
    return job_features

In [24]:
# Fonction qui permet de récupérer les features de chaque métier
def get_job_features_23 (file):
    with open(file, 'r') as f:
        job = json.load(f)
        
    job_features = []
    job_features_ogr = get_job_features_ogr(file)
    for niv1 in comp:
        for niv2 in comp[niv1]:
            for niv3 in comp[niv1][niv2]:
                for niv4 in comp[niv1][niv2][niv3].values():
                    for item in niv4:         
                        if int(item[1]) in job_features_ogr:
                            if niv3=='nan':
                                 job_features.append(niv2)
                            else:
                                job_features.append(niv2 + ' : ' + niv3)
    for niv1 in act:
        for niv2 in act[niv1]:
            for niv3 in act[niv1][niv2]:
                for niv4 in act[niv1][niv2][niv3]:
                    for niv5 in act[niv1][niv2][niv3][niv4].values():
                        for item in niv5:         
                            if int(item[1]) in job_features_ogr:
                                job_features.append(niv2)
    return np.unique(job_features).tolist()

In [25]:
file = dir_path + '/' + 'fiches_metiers/A1101.xml.json'
get_job_features_23 (file)

['Conduire un véhicule, un engin',
 'Contrôler un produit, vérifier une production, un processus',
 'Cultiver, jardiner, exploiter le bois',
 'Engins/véhicules',
 'Entretenir des installations',
 'Juridique, Fiscal et Légal : Environnement',
 'Manipuler un outil, un appareil, un instrument',
 'Matériel/Equipements/Outils/Instruments : Instruments de communication/multimédia',
 'Matériel/Equipements/Outils/Instruments : Machines/outils/appareils',
 'Mécanique générale',
 'Produire, participer à des travaux manuels',
 'Réparer une machine, du matériel',
 'Sciences de la vie et de la terre : Vivant',
 'Sciences de la vie et de la terre : Végétales',
 'Sécurité, Protection des biens et des personnes',
 "Techniques, procédés agricoles et d'entretien des végétaux : Agricole",
 "Techniques, procédés agricoles et d'entretien des végétaux : Espaces Verts",
 'Techniques, procédés de manutention',
 'Techniques, procédés industriels et artisanaux : Métallurgie/ Soudure/ Usinage']

In [10]:
# Fonction qui permet d'obtenir la liste des métiers similaires au métier en argument
def get_sim_jobs(code_rome):
    # Numéro du cluster
    num_cluster = int(clusters.loc[clusters.code_rome=='A1101',].clusters)
    # 
    sim_jobs = []
    for row in clusters.iterrows():
        if row[1]["clusters"]==num_cluster:
            sim_jobs.append(row[1]["intitule"])
    return sim_jobs

In [16]:
# Création d'un dictionnaire avec :
# - code_rome en clés
# - un dictionnaire en valeurs comprenant :
# --> la liste des appellations du métier
# --> la liste des features (ou tags ou variables)
# --> la liste  métiers du même cluster
jobs_ifs = {}
for file in list_files:
    with open(fiches_dir_path + '/' + file, 'r') as f:
        job = json.load(f)

    jobs_ifs[job["code_rome"]] = {}

    # Liste des appelations du métier
    jobs_ifs[job["code_rome"]]["intitule"] = get_job_intitule(fiches_dir_path + '/' + file)    

    # Liste des features du métier
    jobs_ifs[job["code_rome"]]["features"] = get_job_features_23(fiches_dir_path + '/' + file)

    # Liste des métiers similaires
    jobs_ifs[job["code_rome"]]["sim_jobs"] = get_sim_jobs(job["code_rome"])

In [17]:
# Copie du dictionnaire au format JSON
with open(output_file, 'w') as f:
    json.dump(jobs_ifs, f, indent=4)